## Limits
RDP Search does impose limits on the size of the result set when requesting for large data sets.  The following examples provide some useful techniques when dealing with results reaching the upper limits imposed by the backend.

#### NuGet Packages

In [1]:
#r "nuget:Refinitiv.Data.Content, 1.0.0-beta3"
#r "nuget:Microsoft.Data.Analysis"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages Microsoft.Data.Analysis, 0.19.0 Refinitiv.Data.Content, 1.0.0-beta3

Loading extensions from `Microsoft.Data.Analysis.Interactive.dll`

In [2]:
using Newtonsoft.Json.Linq;
using Refinitiv.Data.Content.SearchService;
using Refinitiv.Data.Core;
using Microsoft.Data.Analysis;
using Microsoft.AspNetCore.Html;
using System.Data;
using System;
using Microsoft.DotNet.Interactive.Formatting;
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;

#### Table Output
Helper routine to output data in a table format.

In [3]:
Formatter.Register<DataTable>((dt, writer) =>
{
    if (dt.Columns.Count > 0)
    {
        var rows = new List<ICollection<IHtmlContent>>();

        // Process Each data row
        foreach(var row in dt.Rows.Cast<DataRow>())
        {
            var cells = new List<IHtmlContent>();
            foreach (var cell in row.ItemArray)
            {
                switch (cell)
                {
                    case null:
                        break;
                    case JValue val:
                        cells.Add(td(val.ToString()));
                        break;
                    case IEnumerable<JToken> list:
                        cells.Add(td($"[{string.Join(", ", list.Select(node => $"{node}"))}]"));
                        break;
                    default:
                        cells.Add(td(cell));
                        break;
                }                
            }
            rows.Add(cells);
        }
        
        // Header
        var headers = new List<IHtmlContent>();
        foreach (DataColumn col in dt.Columns)
            headers.Add(th(col.ColumnName) as IHtmlContent);

        var t = table(thead(headers), tbody(rows.Select(r => tr(r))));
        writer.Write(t);
    }
}, "text/html");

In [4]:
// Create a session into the desktop
DesktopSession.Definition().GetSession().OnState((s, state, msg) => Console.WriteLine($"{DateTime.Now}:{msg}. (State: {state})"))
                                        .OnEvent((s, eventCode, msg) => Console.WriteLine($"{DateTime.Now}:{msg}. (Event: {eventCode})"))
                                        .Open();

24/01/2022 11:58:51 AM:DesktopSession. (State: DesktopSession is Pending)
24/01/2022 11:58:51 AM:DesktopSession. (Event: {
  "Contents": "Desktop Session Successfully Authenticated"
})
24/01/2022 11:58:51 AM:DesktopSession. (State: DesktopSession is Opened)


#### Grouping
There may be instances where the result set contains groups of values for properties based on your request.  For example, if I'm interested in retrieving all exchanges within the USA, I can execute this request:

In [5]:
var response = Search.Definition(Search.View.EquityQuotes).Filter("RCSExchangeCountryLeaf eq 'United States'")
                                                          .Top(10000)
                                                          .Select("ExchangeCode, RIC")
                                                          .GetData();
response.Data.Total

4340142

In [6]:
response.Data.Table

IOM,EScv1
IOM,NQcv1
IOM,ESc1
IOM,NQc1
CBT,YMc1
IOM,NKc1
CBF,VXc1
IOM,RTYc1
IOM,ESv1
IOM,DMc1
CBF,VXc2


In the above example, you can see the total available documents is over 4,000,000.  However, due to the nature of the data set, the exchange codes have been repeated which brought back the upper limit of documents within the result set.  **Note**: At the time of this writing, the upper limit has been defined as 10000 result sets.

Instead of performing multiple calls and pulling out the unique codes within each result set, I can apply the grouping features offered by Search to significantly reduce the result set returned.  For example:

In [7]:
response = Search.Definition(Search.View.EquityQuotes).Filter("RCSExchangeCountryLeaf eq 'United States'")
                                                      .Top(10000)
                                                      .Select("ExchangeCode")
                                                      .GroupBy("ExchangeCode") // Exchange codes can be grouped
                                                      .GroupCount(1)           // Then limited to 1 for each to create uniqueness
                                                      .GetData();
response.Data.Table

IOM
CBT
CBF
IMM
NSQ
NYQ
NMQ
NAQ
ASQ
IUS
PNK


As you can see, I've significantly reduced the result set by grouping which now allows the result set using a single API call.  Using the 'grouping' technique to pull out the unique exchange codes is very useful if you wish to return many other properties as part of your results.  However, if you are strictly after the list of exchange codes, the preferred approach is to use Navigators.

#### Navigators
If the goal of your search is to simply capture the list of exchange codes, then the preferred approach in this case is to use Navigators.  A navigator allows the ability to categorize and summarize properties within the result set.  For example, I can provide a simple navigator where I want to bucket all the exchange codes found within the result set.  You can do this using the following request:

In [8]:
response = Search.Definition(Search.View.EquityQuotes).Filter("RCSExchangeCountryLeaf eq 'United States'")
                                                      .Top(0)
                                                      .Navigators("ExchangeCode(buckets:1000)")
                                                      .GetData();

In [9]:
var code = response.Data.Navigators["ExchangeCode"]["Buckets"];
Console.WriteLine($"Total exchange codes found: {code.Count()}");

Total exchange codes found: 149


In [10]:
Console.WriteLine(code);

[
  {
    "Label": "OPQ",
    "Count": 1709078
  },
  {
    "Label": "ONE",
    "Count": 1441126
  },
  {
    "Label": "IOM",
    "Count": 428110
  },
  {
    "Label": "PNK",
    "Count": 71322
  },
  {
    "Label": "CBT",
    "Count": 67005
  },
  {
    "Label": "CME",
    "Count": 37697
  },
  {
    "Label": "OBB",
    "Count": 32916
  },
  {
    "Label": "OTC",
    "Count": 24817
  },
  {
    "Label": "BOS",
    "Count": 19338
  },
  {
    "Label": "THM",
    "Count": 18909
  },
  {
    "Label": "XPH",
    "Count": 16695
  },
  {
    "Label": "MID",
    "Count": 15620
  },
  {
    "Label": "PSE",
    "Count": 15548
  },
  {
    "Label": "NYS",
    "Count": 15158
  },
  {
    "Label": "CIN",
    "Count": 13600
  },
  {
    "Label": "NYQ",
    "Count": 13595
  },
  {
    "Label": "NTV",
    "Count": 13217
  },
  {
    "Label": "BZX",
    "Count": 13109
  },
  {
    "Label": "BYX",
    "Count": 13108
  },
  {
    "Label": "NMS",
    "Count": 11571
  },
  {
    "Label": "NAS",
    "Coun

#### Segmenting the search
When we started with the above search to retrieve the list of exchange codes within the United States, we discovered that the result set returned the entire universe of instruments.  If our goal is to capture the entire instrument list, we cannot group and bucket the result set as we did above.  The # of hits returned is over 4 million so we are forced to go through a tedious process of segmenting the requests.

One way to do this is to choose some kind of indicator that will allow you to group your individual requests to successfully segment the result set.  However, you need to first ask yourself - do I need the entire data universe?  You may only be interested in a specific asset category thus reducing the universe of results significantly.

One possible way to approach this is to first capture the list of asset categories using a navigator on the property: 'RCSAssetCategoryLeaf'.  
For example:

In [11]:
response = Search.Definition(Search.View.EquityQuotes).Filter("RCSExchangeCountryLeaf eq 'United States'")
                                                      .Top(0)
                                                      .Navigators("RCSAssetCategoryLeaf")
                                                      .GetData();
Console.WriteLine(response.Data.Navigators["RCSAssetCategoryLeaf"]["Buckets"]);

[
  {
    "Label": "Equity Cash Option",
    "Count": 1734601
  },
  {
    "Label": "Equity Future",
    "Count": 1460627
  },
  {
    "Label": "Ordinary Share",
    "Count": 415349
  },
  {
    "Label": "Stock Index Future Option",
    "Count": 339182
  },
  {
    "Label": "Stock Index Cash Option",
    "Count": 110116
  },
  {
    "Label": "Equity Future Spread",
    "Count": 51226
  },
  {
    "Label": "Unit",
    "Count": 43784
  },
  {
    "Label": "American Depository Receipt",
    "Count": 30039
  },
  {
    "Label": "Company Warrant",
    "Count": 23560
  },
  {
    "Label": "Preferred Share",
    "Count": 20872
  },
  {
    "Label": "Stock Index Future",
    "Count": 19797
  },
  {
    "Label": "Preference Share",
    "Count": 12760
  },
  {
    "Label": "Depository Receipt",
    "Count": 7996
  },
  {
    "Label": "Depository Share",
    "Count": 7469
  },
  {
    "Label": "Right",
    "Count": 6747
  },
  {
    "Label": "Bond",
    "Count": 6421
  },
  {
    "Label": "Equity

The result of this will not only provide the complete list of categories for you to potentially select the desired ones, but for each, you can see the number of results.  This will further allow you to tune your requests based on these totals.

However, the above summary shows many categories that easily exceed the limits of the server.  If you need to further segment, you can possibly use the ***market cap*** to segment a specific asset category.

For example, let's choose an asset category where we can get a breakdown of the market cap:

In [12]:
// The following navigator will prepare the buckets of evenly distributed market cap ranges such that they fulfill 
// the limit requirements.  Below, I chose 12 as this will produce reasonable buckets we can work with.
var filter = "RCSExchangeCountryLeaf eq 'United States' and RCSAssetCategoryLeaf xeq 'Ordinary Share'";

response = Search.Definition(Search.View.EquityQuotes).Filter(filter)
                                                      .Top(0)
                                                      .Navigators("MktCapTotal(type:range, buckets:12)")
                                                      .GetData();
Console.WriteLine(response.Data.Navigators["MktCapTotal"]["Buckets"]);

[
  {
    "Label": "Below 4131389.97",
    "Filter": "MktCapTotal lt 4131389.97",
    "Count": 10148
  },
  {
    "Label": "Between 4131389.97 And 22327547.74",
    "Filter": "(MktCapTotal ge 4131389.97 and MktCapTotal lt 22327547.74)",
    "Count": 10157
  },
  {
    "Label": "Between 22327547.74 And 63198392.53",
    "Filter": "(MktCapTotal ge 22327547.74 and MktCapTotal lt 63198392.53)",
    "Count": 10140
  },
  {
    "Label": "Between 63198392.53 And 138082113.42",
    "Filter": "(MktCapTotal ge 63198392.53 and MktCapTotal lt 138082113.42)",
    "Count": 10144
  },
  {
    "Label": "Between 138082113.42 And 261859267.37",
    "Filter": "(MktCapTotal ge 138082113.42 and MktCapTotal lt 261859267.37)",
    "Count": 10156
  },
  {
    "Label": "Between 261859267.37 And 436150527.86",
    "Filter": "(MktCapTotal ge 261859267.37 and MktCapTotal lt 436150527.86)",
    "Count": 10187
  },
  {
    "Label": "Between 436150527.86 And 777354379.51",
    "Filter": "(MktCapTotal ge 436150527.86

The first thing to note is that the 'Count' values for each bucket are within the valid limit of the server.  Based on this output, we can use the convenient Filter expressions provided to drive our segmented search requests.

For demonstration purposes, I will select one to retrieve the list of RICs for the specific asset category with the specified market cap range.

In [13]:
// Define our filter
var range1 = response.Data.Navigators["MktCapTotal"]["Buckets"][1]["Filter"];
filter = $"RCSExchangeCountryLeaf eq 'United States' and RCSAssetCategoryLeaf xeq 'Ordinary Share' and {range1}";
filter

RCSExchangeCountryLeaf eq 'United States' and RCSAssetCategoryLeaf xeq 'Ordinary Share' and (MktCapTotal ge 4131389.97 and MktCapTotal lt 22327547.74)

In [14]:
response = Search.Definition(Search.View.EquityQuotes).Filter(filter)
                                                      .Top(0)
                                                      .GetData();
Console.WriteLine($"Request resulted in a segement of {response.Data.Total} documents.");

Request resulted in a segement of 10157 documents.


Based on the buckets I defined, I can now safely use a filter to pull out a segment of instruments.  Despite using a combination of navigators and filters to conveniently define how to break up the segments to avoid these limits, the work to do so is still relatively complicated.

While it may be possible to pull out excessive amounts of data, you should ask yourself if you need to do this.  In most cases, you may be able to reduce the result set when you set up your search instead of pulling in everything then massage the results once you have them in hand.  Search was designed specifically to allow users to filter out unwanted content prior to returning the results.  If you think this way through your searching patterns, you will undoubtedly avoid situations where you need to create complicated algorithms to unnecessarily pull excessive amounts of data. Whether narrowing the request based on interested categories, or data for a specific region, you will find that you can significantly simplify your logic and avoid issues with limits.